In [1]:
import pandas as pd
import numpy as np

In [2]:
df_main = pd.read_csv('clean_data.csv')
df_imdb = pd.read_csv('clean_imdb.csv')

In [3]:
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier, RidgeClassifierCV
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB

In [4]:
np.random.seed(42)
pdf_main = df_main.reindex(np.random.permutation(df_main.index))
pdf_imdb = df_imdb.reindex(np.random.permutation(df_imdb.index))

In [5]:
X_train, X_test, y_train, y_test = pdf_imdb['clean_text'], pdf_main['clean_text'], pdf_imdb['class'], pdf_main['class']

In [6]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Flatten, Conv1D, MaxPooling1D, Dropout, Bidirectional
from keras.optimizers import Adam

Using TensorFlow backend.


In [7]:
import os
embeddings_index = {}
f = open(os.path.join('../', 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [8]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

In [9]:
np.random.seed(42)

In [10]:
EMBEDDING_DIM = 200
MAX_SEQUENCE_LENGTH = 100

In [11]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [12]:
embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

W0726 15:29:51.647759 140107180857152 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [14]:
pad_train = pad_sequences(seq_train, maxlen=100)
pad_test = pad_sequences(seq_test, maxlen=100)

In [18]:
model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(64, dropout=.25, recurrent_dropout=.25, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=.25, recurrent_dropout=.25, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=.25, recurrent_dropout=.25, return_sequences=False)))
# model.add(BLSTM(128, dropout=.25, recurrent_dropout=.25, return_sequences=True))
# model.add(LSTM(128, dropout=.25, recurrent_dropout=.25))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
np.random.seed(42)
model.fit(pad_train, y_train, epochs=5, batch_size=64, validation_data=(pad_test, y_test))

Train on 50000 samples, validate on 2000 samples
Epoch 1/5
50000/50000 [==============================] - 842s 17ms/step - loss: 0.4822 - acc: 0.7691 - val_loss: 0.4441 - val_acc: 0.7810
Epoch 2/5
50000/50000 [==============================] - 817s 16ms/step - loss: 0.3824 - acc: 0.8358 - val_loss: 0.4163 - val_acc: 0.8055
Epoch 3/5
50000/50000 [==============================] - 831s 17ms/step - loss: 0.3455 - acc: 0.8555 - val_loss: 0.3828 - val_acc: 0.8240
Epoch 4/5
50000/50000 [==============================] - 952s 19ms/step - loss: 0.3276 - acc: 0.8626 - val_loss: 0.3844 - val_acc: 0.8270
Epoch 5/5
50000/50000 [==============================] - 1255s 25ms/step - loss: 0.3114 - acc: 0.8695 - val_loss: 0.3554 - val_acc: 0.8430


In [22]:
np.random.seed(42)
model.fit(pad_train, y_train, epochs=3, batch_size=64, validation_data=(pad_test, y_test))

Train on 50000 samples, validate on 2000 samples
Epoch 1/3
50000/50000 [==============================] - 1232s 25ms/step - loss: 0.2925 - acc: 0.8809 - val_loss: 0.3571 - val_acc: 0.8430
Epoch 2/3
50000/50000 [==============================] - 1231s 25ms/step - loss: 0.2797 - acc: 0.8867 - val_loss: 0.3745 - val_acc: 0.8435
Epoch 3/3
50000/50000 [==============================] - 1215s 24ms/step - loss: 0.2719 - acc: 0.8896 - val_loss: 0.3488 - val_acc: 0.8525


In [25]:
# Dont run this
# np.random.seed(42)
# model.fit(pad_train, y_train, epochs=3, batch_size=64, validation_data=(pad_test, y_test))

Train on 50000 samples, validate on 2000 samples
Epoch 1/3
50000/50000 [==============================] - 829s 17ms/step - loss: 0.2600 - acc: 0.8953 - val_loss: 0.3600 - val_acc: 0.8435
Epoch 2/3
50000/50000 [==============================] - 788s 16ms/step - loss: 0.2499 - acc: 0.8997 - val_loss: 0.3647 - val_acc: 0.8490
Epoch 3/3
 4352/50000 [=>............................] - ETA: 6:13 - loss: 0.2267 - acc: 0.9129

KeyboardInterrupt: 

In [23]:
model.save('rnn_deepest_bidir_embed_not_trained.hdf5')

In [26]:
accuracy_score(y_test, model.predict_classes(pad_test))

0.8435